#📌 Extracão

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import json

url = 'https://raw.githubusercontent.com/alura-cursos/challenge2-data-science/refs/heads/main/TelecomX_Data.json'
response = requests.get(url)
data = response.json()
df = pd.json_normalize(data)


#🔧 Transformação

In [5]:
# Eu transformei as colunas que tem apenas yes/no em booleano.
# Pesquisei antes se é uma boa alternativa e aparentemente é
# muito melhor fazer isso pra colunas assim.
def convert_to_int(df):
  for col in df.columns:
    if df[col].dropna().isin(['Yes', 'No', ' yes', ' no', 'YES', 'NO', '']).all():
      df[col] = df[col].astype(str).str.strip().str.capitalize()
      df[col] = df[col].replace('', np.nan)
      df[col] = df[col].map({'Yes': 1, 'No': 0})
      df[col] = df[col].astype('Int64')
  return df

cols = ['internet.InternetService', 'account.Contract', 'account.PaymentMethod']
def categorizar_coluna(df):
  resultado = []
  for col in cols:
    resultado.append(f'Categorias da coluna "{col}": {", ".join(map(str, df[col].unique()))}')
  return '\n'.join(resultado)


print(categorizar_coluna(df))
df = convert_to_int(df)

# converti a coluna em float64
df['account.Charges.Total'] = pd.to_numeric(df['account.Charges.Total'], errors='coerce')
# retirei da coluna churn os valores vazios
df = df[df['Churn'] != '']
# preenchi com o valor 0 os nan desta coluna pois sao clientes novos que
# nao tem cobrança total ainda
df['account.Charges.Total'] = df['account.Charges.Total'].fillna(0)
# como solicitado no trello, adicionei uma coluna nova de cobranças diarias
df['account.Charges.Daily'] = round(df['account.Charges.Monthly'] / (365/12), 2)

# Dicionário de dados

# customerID: número de identificação único de cada cliente
# Churn: se o cliente deixou ou não a empresa
# gender: gênero (masculino e feminino)
# SeniorCitizen: informação sobre um cliente ter ou não idade igual ou maior que 65 anos
# Partner: se o cliente possui ou não um parceiro ou parceira
# Dependents: se o cliente possui ou não dependentes
# tenure: meses de contrato do cliente
# PhoneService: assinatura de serviço telefônico
# MultipleLines: assisnatura de mais de uma linha de telefone
# InternetService: assinatura de um provedor internet
# OnlineSecurity: assinatura adicional de segurança online
# OnlineBackup: assinatura adicional de backup online
# DeviceProtection: assinatura adicional de proteção no dispositivo
# TechSupport: assinatura adicional de suporte técnico, menos tempo de espera
# StreamingTV: assinatura de TV a cabo
# StreamingMovies: assinatura de streaming de filmes
# Contract: tipo de contrato
# PaperlessBilling: se o cliente prefere receber online a fatura
# PaymentMethod: forma de pagamento
# Charges.Monthly: total de todos os serviços do cliente por mês
# Charges.Total: total gasto pelo cliente

Categorias da coluna "internet.InternetService": DSL, Fiber optic, No
Categorias da coluna "account.Contract": One year, Month-to-month, Two year
Categorias da coluna "account.PaymentMethod": Mailed check, Electronic check, Credit card (automatic), Bank transfer (automatic)


#📊 Carga e análise

#📄Relatorio Final